# **IATI Datastore API data collection**

## **Dependencies**

In [2]:
import requests
import csv
import KEYS
import json
from tqdm.notebook import tqdm

## **Define relevant features**

List of all elements found here: https://developer.iatistandard.org/api-details#api=datastore&operation=query 

Description of all elements found here: https://iatistandard.org/en/iati-standard/203/activity-standard/summary-table/

In [6]:
fl_list = [
    "iati_identifier",
    "title_narrative",
    "title_narrative_xml_lang",
    "reporting_org_narrative",
    "reporting_org_ref",
    "description_narrative_xml_lang",
    "description_narrative",
    "recipient_country_code",
    "recipient_region_code",
    "location_name_narrative",
    "activity_status_code",
    "document_link_url",
    "last_updated_datetime",
    "activity_date_type",
    "activity_date_iso_date",
    #"activity_date_narrative",
    "sector_code",
    #"sector_narrative",
    "sector_vocabulary",
    #"tag_code",
    #"tag_narrative",
    #"tag_vocabulary",
    #"policy_marker_code",
    #"policy_marker_narrative",
    #"policy_marker_code",
    #"default_aid_type_code"
]

fl_str = ",".join(fl_list)

## **Fetch data from IATI Datastore**

In [11]:
# REF CODES

# Name | Code
# BMZ | DE-1
# Inter-American Development Bank (IAD) | XI-IATI-IADB
# KfW -> no identifier; probably in BMZ data
# Asian Development Bank (ADB) | XM-DAC-46004
# African Development Bank (ADF) | XM-DAC-46002
#

### Fetch as JSON

In [7]:
url = "https://api.iatistandard.org/datastore/activity/select"
query = {
    "q": 'reporting_org_ref:("XM-DAC-46002")',
    "start": 0, 
    "rows": 1000,  # max 1000 rows per page
    "fl": fl_str
}

headers = {
    "Ocp-Apim-Subscription-Key": KEYS.IATI_KEY
}

try:
    all_docs = []
    page = 0
    pbar = tqdm()

    max_pages = 1
    max_pages_avl = False
    cur_page = -1

    while True:
        if cur_page < max_pages or max_pages_avl == False:
            cur_page += 1
            print(f"Page: {page}")
            response = requests.get(url, params=query, headers=headers)
            
            if response.status_code == 200:
                data = response.json()
                docs = data.get("response", {}).get("docs", [])
                if docs:
                    all_docs.extend(docs)
                    query["start"] += len(docs)
                    page += 1
                    pbar.update(1)
                else:
                    break
            else:
                print("Error:", response.status_code)
                break
        else:
            break

    pbar.close()

    with open("../../src/response_adf.json", mode='w', encoding='utf-8') as file:
        json.dump(all_docs, file, ensure_ascii=False, indent=4)
        
    print("Data downloaded and stored in src!'")

except requests.exceptions.RequestException as e:
    print("Error:", e)

0it [00:00, ?it/s]

Page: 0
Page: 1
Page: 2
Page: 3
Page: 4
Page: 5
Data downloaded and stored in src!'


### Fetch as CSV

In [58]:
url = "https://api.iatistandard.org/datastore/activity/select"
query = {
    "q": "*:*",
    "start": 500000,
    "rows": 1000,
    "fl": fl_str
}

headers = {
    "Ocp-Apim-Subscription-Key": KEYS.IATI_KEY2
}

try:
    response = requests.get(url, params=query, headers=headers)
    if response.status_code == 200:
        data = response.json()
        docs = data.get("response", {}).get("docs", [])
        if docs:
            with open("../data/response_iati.csv", mode='w', newline='', encoding='utf-8') as file:
                writer = csv.DictWriter(file, fieldnames=fl_list)
                writer.writeheader()
                for doc in docs:
                    row = {key: doc.get(key, '') for key in fl_list}
                    writer.writerow(row)
            print("Data downlaoded and stored as 'response_iati.csv'")
        else:
            print("No data in response!")
    else:
        print("Error:", response.status_code)
except requests.exceptions.RequestException as e:
    print("Error:", e)

Data downlaoded and stored as 'response_iati.csv'
